In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Helper function for printing docs


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i + 1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [5]:
!wget https://raw.githubusercontent.com/teddylee777/langchain-kr/refs/heads/main/10-Retriever/data/ai-story.txt -O cohere-sample.txt

--2025-11-17 17:28:10--  https://raw.githubusercontent.com/teddylee777/langchain-kr/refs/heads/main/10-Retriever/data/ai-story.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16574 (16K) [text/plain]
Saving to: ‘cohere-sample.txt’

cohere-sample.txt   100%[===================>]  16.19K  --.-KB/s    in 0.002s  

2025-11-17 17:28:10 (7.50 MB/s) - ‘cohere-sample.txt’ saved [16574/16574]



In [8]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereEmbeddings
from langchain_cohere import ChatCohere
from langchain_cohere import CohereRerank, CohereRagRetriever
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
import os

user_query = "what is Cohere Toolkit?"

# Define the Cohere LLM
llm = ChatCohere(
    cohere_api_key=os.environ["COHERE_API_KEY"], model="command-a-03-2025"
)

# Define the Cohere embedding model
embeddings = CohereEmbeddings(
    cohere_api_key=os.environ["COHERE_API_KEY"], model="embed-english-light-v3.0"
)

# Load text files and split into chunks, you can also use data gathered elsewhere in your application
raw_documents = WebBaseLoader(
    "https://docs.cohere.com/docs/cohere-toolkit"
).load()
text_splitter = CharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0
)
documents = text_splitter.split_documents(raw_documents)

# Create a vector store from the documents
db = Chroma.from_documents(documents, embeddings)

# Create Cohere's reranker with the vector DB using Cohere's embeddings as the base retriever
reranker = CohereRerank(
    cohere_api_key="COHERE_API_KEY", model="rerank-english-v3.0"
)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=reranker, base_retriever=db.as_retriever()
)
compressed_docs = compression_retriever.get_relevant_documents(
    user_query
)
# Print the relevant documents from using the embeddings and reranker
print(compressed_docs)

# Create the cohere rag retriever using the chat model
rag = CohereRagRetriever(llm=llm, connectors=[])
docs = rag.get_relevant_documents(
    user_query,
    documents=compressed_docs,
)
# Print the documents
print("Documents:")
for doc in docs[:-1]:
    print(doc.metadata)
    print("\n\n" + doc.page_content)
    print("\n\n" + "-" * 30 + "\n\n")
# Print the final generation
answer = docs[-1].page_content
print("Answer:")
print(answer)
# Print the final citations
citations = docs[-1].metadata["citations"]
print("Citations:")
print(citations)


ModuleNotFoundError: No module named 'langchain_cohere'

In [7]:
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.readers.web import (
    SimpleWebPageReader,
)  # first, run `pip install llama-index-readers-web`

# create index (we are using an example page from Cohere's docs)
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://docs.cohere.com/v2/docs/cohere-embed"]
)  # you can replace this with any other reader or documents
index = VectorStoreIndex.from_documents(documents=documents)

# create reranker
cohere_rerank = CohereRerank(
    api_key="COHERE_API_KEY", model="rerank-english-v3.0", top_n=2
)

# query the index
query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[cohere_rerank],
)

print(query_engine)

# generate a response
response = query_engine.query(
    "What is Cohere's Embed Model?",
)

print(response)

# To view the source documents
from llama_index.core.response.pprint_utils import pprint_response

pprint_response(response, show_source=True)


ModuleNotFoundError: No module named 'llama_index.postprocessor'